In [ ]:
!pip install numpy opencv-python numba


In [20]:
!pip install Pillow


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls "/content/drive/My Drive"


'Asadullah_SOP_Texas A&M University_Final.docx'
'Case_analysis_export (1).csv'
 Case_analysis_export.csv
 Case_analysis_report_final.csv
 Classroom
 Cleaned_Survey_Data.gsheet
 CNS_Research
'Colab Notebooks'
'Content Creator Simple_april1_2024_nomissingdatatest.csv'
'Content Creator Simple_april6_2024_nomissingdata_engagedresponse (1).gsheet'
'Content Creator Simple_april6_2024_nomissingdata_engagedresponse (2).gsheet'
'Content Creator Simple_april6_2024_nomissingdata_engagedresponse (3).gsheet'
'Content Creator Simple_april6_2024_nomissingdata_engagedresponse.gsheet'
'Content Creator Simple_april6_2024_nomissingdata_engagedresponse.xlsx'
'Content+Creator+Simple_March+22,+2024_09.26 (1).csv'
 Content+Creator+Simple_March+22,+2024_09.26.csv
'Copy of Copy of resumeTemplate.docx'
'Copy of LakshmiSwaminathan_Resume.docx'
'Copy of Lakshmi_Swaminathan_Resume.gdoc'
'Copy of Resume - Abhishek Srikanth - 12 Feb 2024.gdoc'
'Cover Letter_Lakshmi Swaminathan.gdoc'
 Cover_letter_pfw.gdoc
'COVID cer

In [4]:
image_path = '/content/drive/My Drive/Large_image.jpg'

In [26]:
# Increase the maximum number of pixels that can be processed.
Image.MAX_IMAGE_PIXELS = None

In [30]:
import cv2
import time
import numpy as np


def sobel_edge_detection(image):
    height = len(image)
    width = len(image[0])

    # Sobel Kernels
    Gx = [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]
    Gy = [[-1, -2, -1], [0, 0, 0], [1, 2, 1]]

    # Create output image initialized with zeros
    edges = [[0 for _ in range(width)] for _ in range(height)]

    # Apply Sobel filter to each pixel in the original image
    for y in range(1, height - 1):
        for x in range(1, width - 1):
            sumX = sumY = 0
            for i in range(3):
                for j in range(3):
                    ny, nx = y + i - 1, x + j - 1
                    grayscale = image[ny][nx]  # image is already grayscale
                    sumX += grayscale * Gx[i][j]
                    sumY += grayscale * Gy[i][j]

            # Calculate the magnitude of gradients
            mag = int((sumX**2 + sumY**2) ** 0.5)
            # Threshold to keep the value in 0-255 range
            edges[y][x] = min(255, max(0, mag))

    return edges

In [31]:
from PIL import Image

def load_image(image_path):
    """Load an image from the disk and convert to a list of lists (grayscale)."""
    with Image.open(image_path) as img:
        img = img.convert('L')
        return [[img.getpixel((x, y)) for x in range(img.width)] for y in range(img.height)]


In [6]:
def performance_measurement(func, image):
    start_time = time.time()
    result = func(image)
    elapsed_time = time.time() - start_time
    return elapsed_time, result

In [7]:
def sobel_edge_detector_opencv(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
    sobel_magnitude = cv2.sqrt(sobelx**2 + sobely**2)
    sobel_magnitude_uint8 = cv2.convertScaleAbs(sobel_magnitude)
    return sobel_magnitude_uint8

In [32]:
image = load_image(image_path)

In [35]:
python_time, _ = performance_measurement(sobel_edge_detector, image)
print(python_time)

error: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


In [10]:
opencv_time, _ = performance_measurement(sobel_edge_detector_opencv, image_path)
print(opencv_time)


18.05338430404663


In [ ]:

speedup = python_time / opencv_time
efficiency = 100 * (opencv_time / python_time)
cost = python_time  # since it's a serial computation

print("Metric", "Value")
print("------", "-----")
print("Python Time (sec)", f"{python_time:.2f}")
print("OpenCV Time (sec)", f"{opencv_time:.2f}")
print("Speedup", f"{speedup:.2f}")
print("Efficiency (%)", f"{efficiency:.2f}")
print("Cost (seconds)", f"{cost:.2f}")